## Cuda

A general purpose parallel computing platform and programming model


<img align="right"  src="images/cuda1.png" alt="Drawing" style="width: 500px;"/>

<div style="text-align: left"> 

    Grid: Group of blocks executing a kernel
    One grid per CUDA kernel
    
    Block: Group of threads that can be scheduled independently
    Max threads in a block: 1024
    
    Thread: A single context of execution
    
    Kernel: Function that will execute in parallel on multiple threads

</div>


<img src="images/cuda2.png" alt="Drawing" style="width: 700px;"/>


<img align="right"  src="images/cuda3.png" alt="Drawing" style="width: 400px;"/>

<div style="text-align: left"> 

    CUDA Memory Hierarchy:
    
    Per thread local memory: Available only to the single thread, make sure we are writing and reading from per block memory. WHich is faster than global memory
    
    Block shared memory: Shared by all the threads in a block
        Faster than global memory
    
    Global memory: Shared by all blocks and all grids

</div>


## Global memory access

<img src="images/matmul1.png" alt="Drawing" style="width: 800px;"/>


## Improving matrix multiplication

<img align="right"  src="images/matmul_shared.png" alt="Drawing" style="width: 600px;"/>

<div style="text-align: left"> 
    
    Use shared memory to reduce global memory access
    
    Threads in a block work on a tile
    
</div>


In [ ]:
from numba import cuda
print(cuda.gpus)

In [ ]:
from numba import cuda, float32
import numpy 

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

# The data array
A = numpy.full((TPB*2, TPB*3), 3, numpy.float) # [32 x 48] matrix containing all 3's
B = numpy.full((TPB*3, TPB*1), 4, numpy.float) # [48 x 16] matrix containing all 4's

A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)
C_global_mem = cuda.device_array((TPB*2, TPB*1)) # [32 x 16] matrix result

# Configure the blocks
threadsperblock = (TPB, TPB)
blockspergrid_x = int(numpy.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(numpy.ceil(B.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
res = C_global_mem.copy_to_host()

print(res)